In [3]:
from glob import glob
import pandas as pd
import xml.etree.ElementTree as xet
import os
from shutil import copy

In [4]:
path = glob('./data/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:
    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [5]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./data/images\N1.xml,1093,1396,645,727
1,./data/images\N100.xml,134,301,312,350
2,./data/images\N101.xml,31,139,128,161
3,./data/images\N102.xml,164,316,216,243
4,./data/images\N103.xml,813,1067,665,724


In [6]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'./data/images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,./data/images\N1.xml,1093,1396,645,727,./data/images/N1.jpeg,1920,1080
1,./data/images\N100.xml,134,301,312,350,./data/images/N100.jpeg,450,417
2,./data/images\N101.xml,31,139,128,161,./data/images/N101.jpeg,249,239
3,./data/images\N102.xml,164,316,216,243,./data/images/N102.jpeg,478,395
4,./data/images\N103.xml,813,1067,665,724,./data/images/N103.jpeg,1800,1200


In [7]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./data/images\N1.xml,1093,1396,645,727,./data/images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./data/images\N100.xml,134,301,312,350,./data/images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./data/images\N101.xml,31,139,128,161,./data/images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./data/images\N102.xml,164,316,216,243,./data/images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./data/images\N103.xml,813,1067,665,724,./data/images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [8]:
!pip install -r ./yolov5/requirements.txt

In [8]:
# Create a new directory
os.makedirs('./yolov5/data_images/test', exist_ok=True)
os.makedirs('./yolov5/data_images/train', exist_ok=True)

In [9]:
### split the data into train and test
df_train = df.iloc[:200]
df_test = df.iloc[200:]

In [10]:
train_folder = './yolov5/data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

test_folder = './yolov5/data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

In [9]:
!pip install GPUtil

In [10]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [11]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()  

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 10% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  7% | 12% |


In [43]:
!python ./yolov5/train.py --data ./data/data.yaml --cfg ./yolov5/models/yolov5s.yaml --batch-size 8 --name Model --epochs 100

^C


In [ ]:
!python ./yolov5/export.py --weight ./yolov5/runs/train/Model/weights/best.pt --include torchscript onnx